<a href="https://colab.research.google.com/github/aadityasanjay0801/llm-fine-tuning/blob/main/LLM_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fine Tuning BART**
In following notebook, I try to fine tune BART for new summerisation on BBC news and summaries data.



### **Initializing Notebook**

In [ ]:
!pip install ohmeow-blurr==0.0.24 -q
!pip install bert-score -q

In [ ]:
!pip install evaluate -q

In [ ]:
!pip install torchmetrics -q

In [2]:
import pandas as pd
from fastai.text.all import *
import transformers
from blurr.data.all import *
from blurr.modeling.all import *

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `sr

In [3]:
import pandas as pd
import numpy as np

In [49]:
from tqdm.notebook import tqdm_notebook

In [57]:
from bert_score import score
from transformers import AutoModel, AutoTokenizer

In [95]:
import pprint
from torchmetrics.text.bert import BERTScore

# **Preparing Data**

In [5]:
import os

# Define the path to the zipped file
zip_file = "archive.zip"



# Extract the files from the zip archive
with zipfile.ZipFile(zip_file, "r") as zip_ref:
    zip_ref.extractall("tmp")  # Extract contents to temporary directory

In [6]:
# Define the folder paths
news_dir = "tmp/BBC News Summary/News Articles"
summaries_dir = "tmp/BBC News Summary/Summaries"

# Create empty lists to store news and summaries
news_list = []
summaries_list = []
category_list = []

# Loop through each category folder
for category in os.listdir(news_dir):
    # print(f"Category: {category}")
    # Get news and summaries paths
    news_path = os.path.join(news_dir, category)
    summaries_path = os.path.join(summaries_dir, category)



    # Loop through each file in the news folder
    for filename in os.listdir(news_path):
        category_list.append(category)
        # Extract the article number
        article_number = os.path.splitext(filename)[0]
        # print(article_number)
        # Read the news and summary for this article
        news_file = os.path.join(news_path, filename)
        with open(news_file, 'r') as f:
          try:
            news_content = f.read()
          except:
            print(filename)

        summary_file = os.path.join(summaries_path, filename)
        with open(summary_file, 'r') as f:
            summary_content = f.read()

        # Append the news and summary to their respective lists
        news_list.append(news_content)
        summaries_list.append(summary_content)

# Create a Pandas DataFrame
df = pd.DataFrame({"news": news_list, "summary": summaries_list})

# Add the category as a column if desired
df["category"] = category_list

# Print the DataFrame
# print(df)

# Save the DataFrame to a CSV file (optional)
df.to_csv("news_summaries.csv", index=False)

199.txt


# **Loading Model - Pre Trained**

In [22]:

#Import the pretrained model
from transformers import BartForConditionalGeneration
pretrained_model_name = "facebook/bart-large-cnn"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(pretrained_model_name,
                                                                  model_cls=BartForConditionalGeneration)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [23]:
print(hf_model)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

In [26]:
print(hf_config)

BartConfig {
  "_name_or_path": "facebook/bart-large-cnn",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "L

In [27]:
print(hf_tokenizer)


BartTokenizerFast(name_or_path='facebook/bart-large-cnn', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}


#**Create Minibatch and model params**

In [28]:
#Create mini-batch and define parameters
hf_batch_tfm = HF_Seq2SeqBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model,
    task='summarization',
    text_gen_kwargs=
 {'max_length': 248,'min_length': 56,'do_sample': False, 'early_stopping': True, 'num_beams': 4, 'temperature': 1.0,
  'top_k': 50, 'top_p': 1.0, 'repetition_penalty': 1.0, 'bad_words_ids': None, 'bos_token_id': 0, 'pad_token_id': 1,
 'eos_token_id': 2, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'encoder_no_repeat_ngram_size': 0,
 'num_return_sequences': 1, 'decoder_start_token_id': 2, 'use_cache': True, 'num_beam_groups': 1,
 'diversity_penalty': 0.0, 'output_attentions': False, 'output_hidden_states': False, 'output_scores': False,
 'return_dict_in_generate': False, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2, 'remove_invalid_values': False})


In [29]:
print(hf_batch_tfm)

HF_Seq2SeqBeforeBatchTransform:
encodes: (object,object) -> encodes
(object,object) -> encodes
decodes: 


# **PreProcess and Prepare data**

In [36]:
articles = df.head(50)
articles = articles[['news', 'summary']]

In [37]:
articles.head(2)

,news,summary
0,"Fockers retain film chart crown\n\nComedy Meet The Fockers has held on to the number one spot at the North American box office for a second week.\n\nIt took $42.8m (£23.7m) at the weekend, making its overall takings more than $163m (£90m) in 12 days, according to studio estimates. It took $19.1m (£9.9m) on Christmas Day alone, the highest takings on that day in box office history. The sequel to the Ben Stiller comedy Meet The Parents stars Robert De Niro, Dustin Hoffman and Barbra Streisand. The success of Meet the Fockers could help produce record box office revenue for 2004, said Paul De...","Meet the Fockers also broke the box office records for the most money taken on New Year's Eve, when it made $12.2m (£6.2m), and New Year's Day, when it took $18m (£9.4m).It took $19.1m (£9.9m) on Christmas Day alone, the highest takings on that day in box office history.Comedy Meet The Fockers has held on to the number one spot at the North American box office for a second week.However, Christmas takings were down 26.5% on 2003's figures - which was blamed on Christmas falling over a weekend this year.The success of Meet the Fockers could help produce record box office revenue for 2004, sa..."
1,"Lit Idol begins search for author\n\nThe second Pop Idol-style search for literary talent has begun with the help of the brother of Simon Cowell.\n\nWriter Tony Cowell is among the judges who will hear aspiring writers read their work aloud. The winner gets a deal with literary agency Curtis Brown. ""I'm not going to be the Mr Nasty of books,"" said Cowell, 54, in reference to his brother's caustic remarks on TV shows Pop Idol and The X-Factor. The 2004 winner, Paul Cavanagh, went on to sign a deal with Harper Collins.\n\nThis year, the competition is specifically looking for a crime writer....","Writer Tony Cowell is among the judges who will hear aspiring writers read their work aloud.The final five will then have to read their work in front of judging panel.""It's very, very hard to find an agent and extremely difficult, without an agent, to get a publisher to look at your work,"" said Cowell.The winner gets a deal with literary agency Curtis Brown.The 2004 winner, Paul Cavanagh, went on to sign a deal with Harper Collins.""I'm not going to be the Mr Nasty of books,"" said Cowell, 54, in reference to his brother's caustic remarks on TV shows Pop Idol and The X-Factor.The winner will..."


In [38]:
#Prepare data for training
blocks = (HF_Seq2SeqBlock(before_batch_tfm=hf_batch_tfm), noop)
dblock = DataBlock(blocks=blocks, get_x=ColReader('news'), get_y=ColReader('summary'), splitter=RandomSplitter())
dls = dblock.dataloaders(articles, batch_size = 2)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3982: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is depr

# **Training - Finetuning**

## **Defining Metrics**

In [43]:
#Define performance metrics
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'fr' },
            'returns': ["precision", "recall", "f1"]}}


## **Defining Trainer and Training**

In [ ]:

#Model
model = HF_BaseModelWrapper(hf_model)
learn_cbs = [HF_BaseModelCallback]
fit_cbs = [HF_Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

#Specify training
learn = Learner(dls, model,
                opt_func=ranger,loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,splitter=partial(seq2seq_splitter, arch=hf_arch)).to_fp16()

#Create optimizer with default hyper-parameters
learn.create_opt()
learn.freeze()

#Training
learn.fit_one_cycle(3, lr_max=3e-5, cbs=fit_cbs)

# **Trying to get an output**

In [42]:
text_to_generate = df.iloc[-1][0]
print(f"news: {text_to_generate}")
outputs = learn.blurr_generate(text_to_generate, early_stopping=False, num_return_sequences=1)

for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')

news: Kenyon denies Robben Barca return

Chelsea chief executive Peter Kenyon has played down reports that Arjen Robben will return for the Champions League match against Barcelona.

"He's been responding well to treatment and started running on Friday, but we'll have to wait and see," he told BBC Five Live's Sportsweek. "We're looking to getting him back as soon as possible, but he'll be back when it's right for him and for us. "There's no plans at the moment around the Barcelona game." His comments contradict those of chiropractor Jean Pierre Meersseman who treated the Dutchman after he fractured his foot at the start of February. Robben had been expected to be out for six weeks, but Meersseman hinted that the winger could be fit for the vital Stamford Bridge game on 8 March. "I hope he can be back and I will try to help him make that happen," Meersseman told the Mail on Sunday. "I put everything right with Arjen's foot the last time I saw him 12 days ago. It was an obvious correctio

In [44]:
print(outputs)

['Chelsea chief executive Peter Kenyon has played down reports that Arjen Robben will return for the Champions League match against Barcelona. Kenyon denies Robben Barca return on BBC Five Live\'s Sportsweek. His comments contradict those of chiropractor Jean Pierre Meersseman who treated the Dutchman after he fractured his foot at the start of February. Robben had been expected to be out for six weeks, but MeersSEman hinted that the winger could be fit for the vital Stamford Bridge game on 8 March. "We\'re looking to getting him back as soon as possible, but he\'ll be back when it\'s right for him and for us," he said.']


# **Generate data for testing**

In [50]:
test_data_list = []
for index, row in tqdm_notebook(df.iloc[-10:][:2].iterrows()):
    text_to_generate = row['news']
    outputs = learn.blurr_generate(text_to_generate, early_stopping=False, num_return_sequences=1)
    test_data_list.append([row['news'], outputs[0]])




0it [00:00, ?it/s]

In [55]:
test_data = pd.DataFrame(test_data_list, columns =[['True Summary', 'Predicted Summary']])


In [127]:
# test_data.head(2)

##**Calculating BEERTScore over testing dataset**

In [83]:
model_name = "bert-base-uncased"  # You can choose other BERT models
# pretrained_model_name = "facebook/bart-large-cnn"
# model_name = pretrained_model_name

model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [84]:
# print(test_data["True Summary"].values)

In [117]:
# Assuming true_texts and predicted_texts are your lists of texts
true_texts = test_data['True Summary'].values.tolist()
predicted_texts = test_data['Predicted Summary'].values.tolist()



In [118]:
true_texts_formated = [true_text[0] for true_text in true_texts]
predicted_texts_formated = [predicted_text[0] for predicted_text in predicted_texts]


In [126]:
# len(predicted_texts_formated)

In [124]:
bertscore = BERTScore()
calculated_metric = bertscore(predicted_texts_formated, true_texts_formated)

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: The argument `model_name_or_path` was not specified while it is required when the default `transformers` model is used. It will use the default recommended model - 'roberta-large'.
  warnings.warn(*args, **kwargs)  # noqa: B028


In [125]:
pprint.pprint(calculated_metric)

{'f1': tensor([0.9412, 0.9366]),
 'precision': tensor([0.9428, 0.9456]),
 'recall': tensor([0.9395, 0.9279])}
